<a href="https://colab.research.google.com/github/mimilazarova/DD2424-covid-xray-project/blob/master/transfer_efficientNet_Johan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import tf-nightly as tf2
import tensorflow as tf
#print(tf2.__version__)
print(tf.__version__)


2.2.0-rc4


In [4]:
!pip uninstall tensorflow -y
#!pip uninstall tf-nightly
!pip install -U tf-nightly-gpu

Uninstalling tensorflow-2.2.0rc4:
  Successfully uninstalled tensorflow-2.2.0rc4
     |████████████████████████████████| 521.9MB 22kB/s 
     |████████████████████████████████| 460kB 35.7MB/s 
     |████████████████████████████████| 3.0MB 37.2MB/s 


In [0]:
# only run this if you want the system to crash in order to get more RAM
#a = []
#while(1):
#    a.append('1')

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, applications
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

print(tf.__version__)


2.2.0-dev20200508


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
with open('/content/drive/My Drive/Colab Notebooks/data/trainX1.pickle', 'rb') as f:
  X1 = pickle.load(f)
with open('/content/drive/My Drive/Colab Notebooks/data/trainX2.pickle', 'rb') as f:
  X2 = pickle.load(f)

train_X = np.concatenate((X1, X2))

with open('/content/drive/My Drive/Colab Notebooks/data/testX.pickle', 'rb') as f:
  test_X = pickle.load(f)
with open('/content/drive/My Drive/Colab Notebooks/data/train_y.pickle', 'rb') as f:
  train_y = pickle.load(f)
with open('/content/drive/My Drive/Colab Notebooks/data/test_y.pickle', 'rb') as f:
  test_y = pickle.load(f)


In [5]:
N = train_X.shape[0]
testN = test_X.shape[0]

print(N)
print(testN)

train_X = train_X.reshape((N, 224, 224, 1))
test_X = test_X.reshape((testN, 224, 224, 1))

13569
231


In [0]:

N = train_X.shape[0]
testN = test_X.shape[0]

train_X = train_X.reshape((N, 224, 224, 1))
test_X = test_X.reshape((testN, 224, 224, 1))

args = [train_y==0]
X0, vX0, y0, vy0 = train_test_split(train_X[args], train_y[args],  test_size=0.1, random_state=42) 

args = [train_y==1]
X1, vX1, y1, vy1 = train_test_split(train_X[args], train_y[args],  test_size=0.1, random_state=42) 

args = [train_y==2]
X2, vX2, y2, vy2 = train_test_split(train_X[args], train_y[args],  test_size=0.1, random_state=42) 

del vX0, vX1, vX2
del vy0, vy1, vy2

train_X = np.concatenate((X0, X1, X2))
del X0, X1, X2

train_y = np.concatenate((y0, y1, y2))
del y0, y1, y2

#val_X = np.concatenate((vX0, vX1, vX2))
#del vX0, vX1, vX2
#val_y = np.concatenate((vy0, vy1, vy2))
#del vy0, vy1, vy2
#del val_X,  val_y

train_X.shape

In [7]:
class_weights = {}
for c in range(3):
  class_weights[c] = 1000/np.sum(train_y==c)
  print(class_weights[c])


0.13948946854512484
0.2038735983690112
7.352941176470588


In [0]:
#plotting learning curve
def plot_history(history):
  plt.plot(history.history['accuracy'], label='accuracy')
  plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.ylim([0.5, 1])
  plt.legend(loc='lower right')

In [9]:
print(train_X.shape)
print(train_X.shape[1:])

train_X = np.repeat(train_X, 3, axis=3)
test_X = np.repeat(test_X, 3, axis=3)
print(train_X.shape)
print(test_X.shape)
#print(val_X.shape)

(12210, 224, 224, 1)
(224, 224, 1)
(12210, 224, 224, 3)
(231, 224, 224, 3)
(1359, 224, 224, 3)


In [0]:
#del val_X, val_y

In [6]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False, weights='imagenet', input_tensor=None, input_shape=train_X.shape[1:],
    pooling=None, classes=1000, classifier_activation='softmax'
)
base_model.trainable = False

16711680/16705208 [==============================] - 1s 0us/step


In [12]:
base_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
_____________________________________________________________________________________

In [0]:
BATCH_SIZE = 32

image_batch = train_X[:BATCH_SIZE]
feature_batch = base_model(image_batch)
print(feature_batch.shape)

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

prediction_layer = tf.keras.layers.Dense(3)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [9]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

# I tried using 'adam' but no visible changes in accuracy then. So am trying this
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Model)       (None, 7, 7, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
Total params: 4,053,414
Trainable params: 3,843
Non-trainable params: 4,049,571
_________________________________________________________________


In [10]:
len(model.trainable_variables)

2

In [15]:
loss0,accuracy0 = model.evaluate(test_X, test_y)
print(loss0, accuracy0)

8/8 [==============================] - 0s 55ms/step - loss: 1.0596 - accuracy: 0.4329
1.0595641136169434 0.43290042877197266


In [16]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

# change class weights
#class_weights[0] = 1
#class_weights[1] = 1
#class_weights[2] = 4

initial_epochs = 25

history = model.fit(train_X, train_y, 
                           batch_size = 25,
                           epochs=initial_epochs,
                           #class_weight=class_weights,
                           callbacks=[earlystop_callback], 
                           #validation_data=(val_X, val_y))
                           validation_data=(test_X, test_y))

Epoch 1/25
489/489 [==============================] - 18s 37ms/step - loss: 0.7411 - accuracy: 0.5805 - val_loss: 1.1962 - val_accuracy: 0.4329
Epoch 2/25
489/489 [==============================] - 18s 36ms/step - loss: 0.7338 - accuracy: 0.5835 - val_loss: 1.2221 - val_accuracy: 0.4329
Epoch 3/25
489/489 [==============================] - 17s 35ms/step - loss: 0.7350 - accuracy: 0.5848 - val_loss: 1.2280 - val_accuracy: 0.4329
Epoch 4/25
489/489 [==============================] - 17s 35ms/step - loss: 0.7347 - accuracy: 0.5843 - val_loss: 1.2566 - val_accuracy: 0.4329
Epoch 5/25
489/489 [==============================] - 17s 35ms/step - loss: 0.7331 - accuracy: 0.5850 - val_loss: 1.2305 - val_accuracy: 0.4329
Epoch 6/25
489/489 [==============================] - 17s 35ms/step - loss: 0.7338 - accuracy: 0.5844 - val_loss: 1.2746 - val_accuracy: 0.4329
Epoch 7/25
489/489 [==============================] - 17s 35ms/step - loss: 0.7330 - accuracy: 0.5808 - val_loss: 1.2519 - val_accuracy:

In [0]:
plot_history(history)

pred = efficientNet.predict(test_X)
pred_1 = np.argmax(pred, axis=1)

conf_matrix = confusion_matrix(test_y, pred_1, labels=[0, 1, 2])
print("Confusion matrix:")
print(conf_matrix)
print(classification_report(test_y, pred_1, digits=3))

# Fine-tuning the pre-trained model

In [11]:
base_model.trainable = True

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 200

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  237


In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate)/10,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

model.summary()

In [0]:
len(model.trainable_variables)

In [0]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_X,
                         train_y,
                         batch_size=25,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=(test_X, test_y))

In [0]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plot_history(history_fine)